In [ ]:
import os
import numpy as np

# Import tensorflow before transformers with logs deactivated to avoid printing tf logs
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import tensorflow as tf
if not tf.executing_eagerly():
    tf.compat.v1.enable_eager_execution()

from PIL import Image
from pycocotools.coco import COCO
import torch
from torch.utils.data import DataLoader, Dataset
import torch.nn.functional as F
from transformers import SegformerForSemanticSegmentation, SegformerFeatureExtractor
from tqdm import tqdm
import evaluate

In [ ]:
target_classes = torch.tensor([5, 6, 7])
NUM_CLASSES = 2

In [ ]:
class CocoFilteredMaskDataset(Dataset):
    def __init__(self, image_dir, annotation_file, extractor, target_classes):
        """
        Args:
            image_dir (str): Directory containing images.
            annotation_file (str): Path to COCO-format JSON file.
            extractor: Segformer feature extractor.
            target_classes (torch.Tensor or list[int]): List or tensor of category IDs to keep as foreground (label=1).
        """
        self.image_dir = image_dir
        self.extractor = extractor
        self.coco = COCO(annotation_file)
        self.image_ids = list(self.coco.imgs.keys())

        # Store target category IDs as a tensor for fast comparison
        self.target_cat_ids = torch.tensor(target_classes) if not isinstance(target_classes, torch.Tensor) else target_classes

    def __len__(self):
        return len(self.image_ids)

    def __getitem__(self, idx):
        image_id = self.image_ids[idx]
        img_info = self.coco.loadImgs(image_id)[0]
        image_path = os.path.join(self.image_dir, img_info["file_name"])
        image = Image.open(image_path).convert("RGB")

        height, width = img_info["height"], img_info["width"]
        mask = np.zeros((height, width), dtype=np.uint8)

        ann_ids = self.coco.getAnnIds(imgIds=image_id)
        anns = self.coco.loadAnns(ann_ids)

        for ann in anns:
            if ann["iscrowd"] == 0:
                ann_mask = self.coco.annToMask(ann)
                category_id = ann["category_id"]
                # Apply the category_id to the mask (overwrite existing values)
                mask[ann_mask == 1] = category_id

        encoded = self.extractor(image, return_tensors="pt")

        return {
            "pixel_values": encoded["pixel_values"].squeeze(0),
            "labels": torch.tensor(mask, dtype=torch.long),
            "image": image
        }

In [ ]:
def load_coco_split(dataset_dir, split_name, feature_extractor):
    image_dir = os.path.join(dataset_dir, split_name)
    annotation_path = os.path.join(dataset_dir, split_name, "_annotations.coco.json")
    dataset = CocoFilteredMaskDataset(image_dir, annotation_path, feature_extractor, target_classes)
    return DataLoader(dataset, batch_size=1, shuffle=False, collate_fn=lambda x: x)


In [ ]:
def predict(model, extractor, image, device):
    pixel_values = extractor(image, return_tensors='pt').pixel_values.to(device)
    with torch.no_grad():
        logits = model(pixel_values).logits
        logits = F.interpolate(
            logits,
            size=image.size[::-1],  # (width, height) → (height, width)
            mode='bilinear',
            align_corners=False
        )
    labels = torch.argmax(logits.squeeze(), dim=0)
    return labels

In [ ]:
def evaluate_split(model, loader, split_name, feature_extractor, device):
    metric = evaluate.load("mean_iou")
    for batch in tqdm(loader, desc="Evaluating"):
        sample = batch[0]
        image = sample["image"]
        label = sample["labels"].to(device)

        prediction = predict(model, feature_extractor, image, device)
        binary_prediction = torch.isin(prediction, target_classes).long()
        predictions = binary_prediction.unsqueeze(0).cpu().numpy()
        references = label.unsqueeze(0).numpy()

        tqdm.write(f"predictions shape: {predictions.shape}")
        tqdm.write(f"references shape: {references.shape}")

        metric.add_batch(
            predictions=predictions,
            references=references
        )
        tqdm.write(str(metric.compute(num_labels=2, ignore_index=None)))

    results = metric.compute(num_labels=NUM_CLASSES)
    print(f"\n{split_name.capitalize()} Metrics:")
    for k, v in results.items():
        print(f"{k}: {v}")

In [ ]:
## Load segformer model for evaluation
model_name = "nvidia/segformer-b0-finetuned-cityscapes-1024-1024"
model = SegformerForSemanticSegmentation.from_pretrained(model_name)
feature_extractor = SegformerFeatureExtractor.from_pretrained(model_name)


In [ ]:
dataset_dir = os.path.join(
    os.path.dirname((os.getcwd())), "dataset/segmentation/waymo_landmark_segmentation.v1-v1_poles.coco-segmentation"
)

In [ ]:
train_loader = load_coco_split(dataset_dir, "train", feature_extractor)
val_loader = load_coco_split(dataset_dir, "valid", feature_extractor)


In [ ]:
evaluate_split(model, train_loader, "train", feature_extractor, 'cpu')

In [ ]:
evaluate_split(model, val_loader, "valid", feature_extractor, 'cpu')
